# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import os
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
import re
import tensorflow as tf
from models.llama3.generation import Llama

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

In [3]:
import appdirs 
print(appdirs.user_data_dir("Open Interpreter"))


/Users/pranitdas/Library/Application Support/Open Interpreter


# Question Classifier

## Text-Classification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("uw-vta/bloominzer-0.1")
model = AutoModelForSequenceClassification.from_pretrained("uw-vta/bloominzer-0.1")
bloominzer = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
print(bloominzer("If I have 2 pair of apple, can i make apple pie with it?"))

Device set to use mps:0


[{'label': 'Synthesis', 'score': 0.9990537762641907}]


## LLM Classification

## Zero-Shot

### BART

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

bart = pipeline("zero-shot-classification",
                      model=model , tokenizer=tokenizer)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use mps:0


In [ ]:
sequence_to_classify = "If I have 2 pair of apple, can i make apple pie with it?"
candidate_labels = ['knowledge', 'comprehension', 'application', 'analysis','synthesis', 'evaluation']
label = bart(sequence_to_classify, candidate_labels)

In [15]:
label['labels'][0]

'application'

### mDeBERTa-v3-base-mnli-xnli

In [15]:
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

ya_classifier = pipeline("zero-shot-classification",
                      model=model , tokenizer=tokenizer)

Device set to use mps:0


In [16]:
sequence_to_classify = "If I have 2 pair of apple, can i make apple pie with it?"
candidate_labels = ['knowledge', 'comprehension', 'application', 'analysis','synthesis', 'evaluation']
hypothesis_template = "This text is about blooms taxonomy and it is classified as {}."
label = ya_classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)

In [17]:
label['labels'][0]

'knowledge'

## Text Generation

### LLAMA

In [7]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="modularai/Llama-3.1-8B-Instruct-GGUF",
	filename="llama-3.1-8b-instruct-q4_k_m.gguf"
)


llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 17592186040973 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /Users/pranitdas/.cache/huggingface/hub/models--modularai--Llama-3.1-8B-Instruct-GGUF/snapshots/966694508430d1177f6d585de779250e7a34bc3a/./llama-3.1-8b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llam

In [10]:
llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "determine blooms level of query 'What is the capital of France?'"
		}
	]
)

Llama.generate: 48 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    2249.13 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   20879.87 ms /   463 runs   (   45.10 ms per token,    22.17 tokens per second)
llama_perf_context_print:       total time =   21024.56 ms /   464 tokens


{'id': 'chatcmpl-aec2c46b-09b5-4b06-8ce9-b806d3dfaa04',
 'object': 'chat.completion',
 'created': 1748159390,
 'model': '/Users/pranitdas/.cache/huggingface/hub/models--modularai--Llama-3.1-8B-Instruct-GGUF/snapshots/966694508430d1177f6d585de779250e7a34bc3a/./llama-3.1-8b-instruct-q4_k_m.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'To determine the Bloom filter\'s level of query \'What is the capital of France?\' we need to consider the following:\n\n1. The query is a specific question about a well-known fact.\n2. The query is not a general or vague question.\n3. The query is not a question about a common misspelling or variation of a word.\n\nBased on these considerations, I would rate the Bloom filter\'s level of query as follows:\n\n- **Specificity**: High (9/10) - The query is very specific and asks for a specific piece of information.\n- **Relevance**: High (9/10) - The query is highly relevant to the topic of geography and France.\n- **Uniq

### Gemini

# Query Augmentation